In [1]:
import pandas as pd
import numpy as np
import os

In [17]:
os.listdir('../rs')

['.ipynb_checkpoints',
 'itemCF.ipynb',
 'test.csv',
 'train.csv',
 'user_item_dwelling time.ipynb']

In [41]:
train = pd.read_csv('../rs/train.csv')
test = pd.read_csv('../rs/test.csv')

train['timestamp'] = pd.to_datetime(train['timestamp'],unit='s')
test['timestamp'] = pd.to_datetime(test['timestamp'],unit='s')

In [42]:
df = pd.concat([train, test])
del train, test
df.shape

(19715327, 12)

In [44]:
df = df.sort_values(['user_id','session_id','timestamp']).reset_index(drop = True)
df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,0001VQMGUI65,466687f1875ee,2018-11-01 12:38:49,1,interaction item deals,3133074,BR,"Brasília, Brazil",desktop,NaN,NaN,NaN
1,0001VQMGUI65,466687f1875ee,2018-11-01 12:39:09,2,interaction item image,3133074,BR,"Brasília, Brazil",desktop,NaN,NaN,NaN
2,0001VQMGUI65,466687f1875ee,2018-11-01 12:39:28,3,clickout item,3133074,BR,"Brasília, Brazil",desktop,NaN,3133074|2653226|477811|164667|1990735|104972|1...,43|55|49|51|44|43|47|47|55|46|31|24|33|75|52|4...
3,0001VQMGUI65,466687f1875ee,2018-11-01 12:39:34,4,interaction item image,3133074,BR,"Brasília, Brazil",desktop,NaN,NaN,NaN
4,0001VQMGUI65,466687f1875ee,2018-11-01 12:39:44,5,interaction item deals,3133074,BR,"Brasília, Brazil",desktop,NaN,NaN,NaN


In [58]:
reference_first = np.zeros(len(df))
reference_first[0] = 1
check = df['reference'][0]

for i in range(1, len(df)) :
    if check != df['reference'][i] :
        check = df['reference'][i]
        reference_first[i] = 1

In [62]:
reference_first = pd.Series(reference_first)
reference_first.to_pickle('../rs/reference_first.pkl')

In [ ]:
dwell_df = pd.DataFrame(columns = ['user_id','reference','dwell_time'])
dwell_df['reference'] = df[reference_first == 1]['reference']
dwell_df.reset_index(drop = True, inplace = True)

first_time = df[reference_first == 1]['timestamp']
first_time.reset_index(drop = True, inplace = True)

last_time = pd.Series(np.roll(first_time,-1))

dwell_df['dwell_time'] = (last_time - first_time).astype('timedelta64[s]')

In [22]:
df.groupby(['user_id', 'session_id'])

In [ ]:
test.groupby(['user_id'])

In [24]:
df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
176,03P4VFKK12UO,325fafb5fa450,2018-11-01 21:25:00,54,clickout item,65685,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,1306936|56482|2842358|6881276|65685|63259|6539...,178|104|110|94|57|96|46|61|48|35|50|38|59|44|5...
177,03P4VFKK12UO,325fafb5fa450,2018-11-01 21:25:38,55,clickout item,1320460,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,1306936|56482|2842358|6881276|65685|63259|6539...,178|104|110|94|57|96|46|61|48|35|50|38|59|44|5...
445,0NA8E4AD2VY7,336d9a46b826f,2018-11-01 13:20:52,91,clickout item,1091180,RU,"Yekaterinburg, Russia",desktop,Free WiFi (Combined)|Breakfast Included,1045290|3135752|1372961|1369024|3384308|137107...,44|87|30|62|73|59|85|43|68|31|35|48|59|56|42|1...
446,0NA8E4AD2VY7,336d9a46b826f,2018-11-01 13:21:33,92,clickout item,1091180,RU,"Yekaterinburg, Russia",desktop,Free WiFi (Combined)|Breakfast Included,1045290|3135752|1372961|1369024|3384308|137107...,44|87|30|62|73|59|85|43|68|31|35|48|59|56|42|1...
466,0NA8E4AD2VY7,336d9a46b826f,2018-11-01 13:31:47,112,clickout item,3404929,RU,"Yekaterinburg, Russia",desktop,Free WiFi (Combined)|Breakfast Included,1045290|3135752|1372961|1369024|3384308|137107...,44|87|30|62|73|59|85|43|68|31|35|48|59|56|42|1...
